In [1]:
import sqlalchemy as sql
import pandas as pd
import json

In [2]:
with open("../config.json", "r") as file:
    config = json.loads(file.read())
engine = sql.create_engine(config['db_address'].replace("test", "finance"))
dates = pd.date_range("2020-06-01", "now").astype(str)

In [3]:
options = []
chunk_size = 10
for i in range(chunk_size, len(dates) + chunk_size, chunk_size):
    _dates = tuple(dates[i - chunk_size : i])
    query = f"""
        SELECT
            options.*,
            ohlc.adjclose_price as stock_price,
            ohlc.dividend_yield
        FROM
            options
        INNER JOIN
            ohlc
            USING (date_current, ticker)
        WHERE
            ticker = "SPY"
        AND date_current IN {_dates}
    """
    print("Fetching Dates", _dates)
    options.append(pd.read_sql(query, engine))
options = pd.concat(options).reset_index(drop=True)

Fetching Dates ('2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-06', '2020-06-07', '2020-06-08', '2020-06-09', '2020-06-10')
Fetching Dates ('2020-06-11', '2020-06-12', '2020-06-13', '2020-06-14', '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19', '2020-06-20')
Fetching Dates ('2020-06-21', '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26', '2020-06-27', '2020-06-28', '2020-06-29', '2020-06-30')
Fetching Dates ('2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10')
Fetching Dates ('2020-07-11', '2020-07-12', '2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16', '2020-07-17', '2020-07-18', '2020-07-19', '2020-07-20')
Fetching Dates ('2020-07-21', '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-25', '2020-07-26', '2020-07-27', '2020-07-28', '2020-07-29', '2020-07-30')
Fetching Dates ('2020-07-31', '2020-08-01', '2020-08-02', '2020-

In [4]:
query = f"""
    SELECT
        *
    FROM
        treasuryrates
    WHERE
        date_current IN {tuple(dates)}
"""
rates = pd.read_sql(query, engine)

In [5]:
options.to_csv("data/options.csv", index=False)
rates.to_csv("data/rates.csv", index=False)

In [6]:
query = f"""
    SELECT
        *
    FROM
        treasuryratemap
    WHERE
        date_current = "2020-06-01"
"""
ratemap = pd.read_sql(query, engine)
ratemap.to_csv("data/ratemap.csv", index=False)